In [1]:
from pymavlink import mavutil
import time
from datetime import datetime, timezone
# from influxdb import InfluxDBClient
import math


In [2]:
master = mavutil.mavlink_connection('udp:localhost:14551')
master.wait_heartbeat()
time.sleep(2)

In [95]:
class MavlinkData:
    def __init__ (self, mavlink_data):
        if (mavlink_data is None):
            self.data = None
            return None

        self.type_capture = ['AHRS3','AIRSPEED_AUTOCAL','AOA_SSA','ATTITUDE','BATTERY2','EKF_STATUS_REPORT','RC_CHANNELS','SERVO_OUTPUT_RAW','VFR_HUD','WIND','POWER_STATUS','NAV_CONTROLLER_OUTPUT','HEARTBEAT','GLOBAL_POSITION_INT','GPS_RAW_INT','GPS2_RAW','BATTERY_STATUS','SYS_STATUS','SYSTEM_TIME','PARAM_VALUE']

        #self.type_special = ['NAMED_VALUE_FLOAT']

        if mavlink_data._type in self.type_capture:
            self.data = mavlink_data
        else:
            self.data = None

    def MavlinkData(self):
        return self.data

    def _process_battery_status(self):

        # print(self)
        data_dict = {}
        data_dict["voltage_1"] = (((int(self.data.voltages[0]) & 0xFF00) >> 8)+220)/100.0 # Battery Cell Voltage | [V]
        data_dict["voltage_2"] = (((int(self.data.voltages[1]) & 0xFF00) >> 8)+220)/100.0 # Battery Cell Voltage | [V]
        data_dict["voltage_3"] = (((int(self.data.voltages[2]) & 0xFF00) >> 8)+220)/100.0 # Battery Cell Voltage | [V]
        data_dict["voltage_4"] = (((int(self.data.voltages[0]) & 0x00FF)     )+220)/100.0 # Battery Cell Voltage | [V]
        data_dict["voltage_5"] = (((int(self.data.voltages[1]) & 0x00FF)     )+220)/100.0 # Battery Cell Voltage | [V]
        data_dict["voltage_6"] = (((int(self.data.voltages[2]) & 0x00FF)     )+220)/100.0 # Battery Cell Voltage | [V]

        # MPPT
        data_dict["charger_current"]  = (int(self.data.voltages[3]) & 0x00FF)/10          # 0xEDD7 | Charger Current | [A]
        data_dict["charger_voltage"]  = (int(self.data.voltages[3]) & 0x07FF)/100         # 0xEDD5 | Charger Voltage | [V]
        data_dict["charger_power"]  = data_dict["charger_current"] * data_dict["charger_voltage"] # Calculated Value | [W]
        

        # Solar Array 
        data_dict["panel_voltage"]    = (int(self.data.voltages[4]) & 0x03FF)/100           # 0xEDBB | Panel Voltage | [A]
        data_dict["panel_power"]      = (int(self.data.current_battery))/100                # 0xEDBC | Panel Power   | [W]
        
        if data_dict["panel_voltage"] != 0:
            data_dict["panel_current"] = data_dict["panel_power"]/data_dict["panel_voltage"] # Calculated Value | [A]
        else:
            data_dict["panel_current"] = float('Inf')

        # Battery
        # data_dict["battery_voltage"]  = sum(self.data.voltages[0:6])/1000
        # data_dict["battery_current"]  = (int(self.data.current_battery)/100) - data_dict["charge_current"]
        # data_dict["battery_power"]    = data_dict["battery_voltage"] * data_dict["battery_current"]
        # Load

        # Temperature Sensors
        wCurrentConsumed = int(self.data.current_consumed) & 0xFFFF
        wIdx = int((wCurrentConsumed & 0xF800) >> 11)
        wTemperature =  0.1 * round((wCurrentConsumed & 0x07FF)) - 50.0
        if wIdx == 0:
            data_dict["temperature_mppt"] = wTemperature

        data_dict["index"] = wIdx
        data_dict["temperature"] = wTemperature

        # print(data_dict)
        if wIdx < 5:
            print(data_dict["index"], data_dict["temperature"])

        return data_dict

    def to_influx_db(self):
        if self.data is None:
            return None
        else:
            dict_body = {}
            dict_body["measurement"] = self.data._type
            # local time in RFC 3339 format
            dict_body["time"]        = datetime.now(timezone.utc).astimezone().isoformat()
            dict_body["fields"]      = self.data.to_dict()
            
            if self.data._type == 'PARAM_VALUE':
                dict_body["tags"] = {"param_id": dict_body["fields"]["param_id"]}

            if self.data._type == 'BATTERY_STATUS':
                if (self.data.id == 1):
                    dict_body["fields"] = self._process_battery_status()
                    dict_body["measurement"] = "CREATEV_ELEC"
                    #print((self.data.current_battery))
                    #print(dict_body)
                else:
                    dict_body["fields"]["voltages"] = dict_body["fields"]["voltages"][0]
                    dict_body["measurement"] = "BATTERY_STATUS_"+str(self.data.id)


            return [dict_body]

while True:
    # Get data from mavlink
    mavlink_data = MavlinkData(master.recv_msg())
    mavlink_data.to_influx_db()
    # wData = mavlink_data.data

    # try:
    #     if (wData._type == 'BATTERY_STATUS') & (wData.id == 1):
    #         readBatteryStatus(wData)
    # except:
    #     continue
    
    # print(mavlink_data)
    # influxdb_point = mavlink_data.to_influx_db()
    # if influxdb_point is not None:
    #     client.write_points(influxdb_point)
        #print(influxdb_point)

2021-03-24T00:37:38.308763-04:00 0 806 30.60000000000001
2021-03-24T00:37:38.311762-04:00 2 4854 25.799999999999997
2021-03-24T00:37:40.011234-04:00 0 806 30.60000000000001
2021-03-24T00:37:40.257169-04:00 2 4854 25.799999999999997
2021-03-24T00:37:41.918624-04:00 0 806 30.60000000000001
2021-03-24T00:37:42.010606-04:00 1 2807 25.900000000000006
2021-03-24T00:37:42.258521-04:00 3 6903 25.900000000000006
2021-03-24T00:37:43.920963-04:00 1 2807 25.900000000000006
2021-03-24T00:37:44.012156-04:00 2 4854 25.799999999999997
2021-03-24T00:37:44.262084-04:00 4 8692 0.0
2021-03-24T00:37:45.518196-04:00 1 2806 25.799999999999997
2021-03-24T00:37:47.263172-04:00 0 806 30.60000000000001
2021-03-24T00:37:47.499071-04:00 3 6903 25.900000000000006
2021-03-24T00:37:49.124040-04:00 0 806 30.60000000000001
2021-03-24T00:37:49.620111-04:00 4 8692 0.0
2021-03-24T00:37:51.130106-04:00 2 4854 25.799999999999997
2021-03-24T00:37:52.633125-04:00 1 2807 25.900000000000006
2021-03-24T00:37:54.630078-04:00 1 28

KeyboardInterrupt: 

In [ ]:
master.close()

In [ ]:
# mlog.rewind()
# while (True):
#     # Get data from mavlink
#     mavlink_data = MavlinkData(mlog.recv_msg())
#     #mavlink_data.to_influx_db()
#     influxdb_point = mavlink_data.to_influx_db()

#     if influxdb_point is not None:
#         influxdb_point[0]["time"] = datetime.utcfromtimestamp(mlog.timestamp).isoformat()
#         client.write_points(influxdb_point)


In [ ]:
while (True):
#     # Get data from mavlink
    mavlink_data = master.recv_match(type='PARAM_VALUE')
    if mavlink_data is not None:
        print(mavlink_data)